### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Colby Alexander Hoke
# UNC Data Analytics Bootcamp, June 2020
# CC-BY-SA
# --------------------------------------
# INPUT:
# School CSV, formatted as: school ID, school_name, type, size, budget
# Student CSV, formatted as: student ID, student_name, gender, grade, school_name, reading_score, math_score
# 
# RETURN: Several analyses, detailed below
# --------------------------------------

# Import dependencies
import pandas as pd

# Load csv files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read school and student data files, then store into separate pandas dataframes
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merge the dataframes
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name"])

# Display the merged dataframe
school_data_complete


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# --------------------------------------
# Summarize the school district
# --------------------------------------

# Count unique schools
# Sum all students—some may have the same name, so use count()
# Sum all school budgets—same names would duplicate budgets, so use unique() then sum()
tot_schools = school_data_complete["school_name"].nunique()
tot_students = school_data_complete["student_name"].count()
tot_budget = school_data_complete["budget"].unique().sum()

# Get mean math score & round to 2 decimal places
# Get mean reading score & round to 2 decimal places
avg_math = round(school_data_complete["math_score"].mean(),2)
avg_reading = round(school_data_complete["reading_score"].mean(),2)

# Get % of math scores >= 70 (passing) & round to 2 decimal places
# Get % of reading scores >= 70 (passing) & round to 2 decimal places
pct_math_pass = round(school_data_complete[school_data_complete.math_score >= 70].count()["math_score"]/tot_students*100,2)
pct_reading_pass = round(school_data_complete[school_data_complete.reading_score >= 70].count()["reading_score"]/tot_students*100,2)

# Get % of students who pass both math + reading (>= 70 for both) & round to 2 decimal places
pct_overall_pass = round(school_data_complete[(school_data_complete.math_score >= 70) & (school_data_complete.reading_score >= 70)].count()["student_name"]/tot_students*100,2)

# Summarize this data
summary_df = pd.DataFrame({
    "Total Schools": tot_schools,
    "Total Students": tot_students,
    "Total Budget":tot_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_reading,
    "% Passing Math": pct_math_pass,
    "% Passing Reading": pct_reading_pass,
    "% Overall Passing": pct_overall_pass},index=[0])

# Format series to look cleaner
summary_df["Total Budget"] = summary_df["Total Budget"].map("${:,.2f}".format)
summary_df["Total Students"] = summary_df["Total Students"].map("{:,}".format)

# Display the summary dataframe
summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# --------------------------------------
# Summarize each school
# *To match the image in the assignment, we wouldn't round to 2 decimal places
# *However, that's inconsistent reporting, so I left it in
# --------------------------------------

# Group data by individual school
school_data_grouped = school_data_complete.groupby("school_name")

# Get school's type & make it a string
school_type = school_data_grouped["type"].unique().str[0]

# Count students per school, get each school's budget, then divide school budget by students
school_students = school_data_grouped["student_name"].count()
school_budget = school_data_grouped["budget"].unique().astype(float)
per_student_budget = school_budget / school_students

# Get each school's students' math average & round to 2 decimal places
# Get each school's students' reading average & round to 2 decimal places
school_math_avg = round(school_data_grouped["math_score"].mean(),2)
school_reading_avg = round(school_data_grouped["reading_score"].mean(),2)

# Create a new dataframe to hold all math scores >= 70
# Group new dataframe by school
# Get % of a school's students that pass math & round to 2 decimal places
math_pass_all = school_data_complete[(school_data_complete["math_score"]>=70)]
school_math_pass = math_pass_all.groupby("school_name")
school_math_pass_pct = round(school_math_pass["math_score"].count() / school_students*100,2)

# Create a new dataframe to hold all reading scores >= 70
# Group new dataframe by school
# Get % of a school's students that pass reading & round to 2 decimal places
reading_pass_all = school_data_complete[(school_data_complete["reading_score"]>=70)]
school_reading_pass = reading_pass_all.groupby("school_name")
school_reading_pass_pct = round(school_reading_pass["reading_score"].count() / school_students*100,2)

# Create a new dataframe to hold all math & reading scores >= 70
# Group new dataframe by school
# Get % of a school's students that pass math + reading & round to 2 decimal places
mathreading_pass_all = school_data_complete[(school_data_complete["math_score"]>=70)&(school_data_complete["reading_score"]>=70)]
school_mathreading_pass = mathreading_pass_all.groupby("school_name")
school_mathreading_pass_pct = round(school_mathreading_pass["math_score"].count() / school_students*100,2)

# Build a dataframe with our values
school_summary_df = pd.DataFrame({
    "School type": school_type,
    "Total Students": school_students,
    "Total School Budget":school_budget,
    "Per Student Budget":per_student_budget,
    "Average Math Score": school_math_avg,
    "Average Reading Score": school_reading_avg,
    "% Passing Math": school_math_pass_pct,
    "% Passing Reading": school_reading_pass_pct,
    "% Overall Passing": school_mathreading_pass_pct
    })

# Format the dataframe
# Create a copy, since .map() will convert some series to objects & we may need numbers later
# Format series to look cleaner
school_summary_df_formatted = school_summary_df.copy()
school_summary_df_formatted["Total School Budget"] = school_summary_df_formatted["Total School Budget"].map("${:,.2f}".format)
school_summary_df_formatted["Per Student Budget"] = school_summary_df_formatted["Per Student Budget"].map("${:,.2f}".format)

# Reset the index to remove school_name
school_summary_df_formatted.index.name = None

# Display the dataframe
school_summary_df_formatted


,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# --------------------------------------
# Summarize top 5 schools, based on overall performance, & display it
# --------------------------------------

school_summary_df_formatted.sort_values(by=["% Overall Passing"],ascending=False).head(5)


,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# --------------------------------------
# Give a summary of the bottom 5 schools, based on overall performance, & display it
# --------------------------------------

school_summary_df_formatted.sort_values(by=["% Overall Passing"]).head(5)


,School type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

#Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
  * Create a pandas series for each grade. Hint: use a conditional statement.
  * Group each series by school
  * Combine the series into a dataframe  
  * Optional: give the displayed data cleaner formatting

In [6]:
# --------------------------------------
# Summarize math scores, based on grade (9, 10, 11, 12)
# *To match the image in the assignment, we wouldn't round to 2 decimal places
# *However, that's inconsistent reporting, so I left it in
# --------------------------------------

# Make a new dataframe, based on original data
# Keep only the school_name, grade, & math_score series
math_by_grade = school_data_complete[["school_name", "grade", "math_score"]]
math_by_grade.set_index("school_name")

# Separate 9th grade math, group by school, then calculate & round average
nineth_math = math_by_grade[(math_by_grade["grade"] == "9th")]
nineth_math_grouped = nineth_math.groupby("school_name")
nineth_math_avg = round(nineth_math_grouped.mean(),2)

# Separate 10th grade math, group by school, then calculate & round average
tenth_math = math_by_grade[(math_by_grade["grade"] == "10th")]
tenth_math_grouped = tenth_math.groupby("school_name")
tenth_math_avg = round(tenth_math_grouped.mean(),2)

# Separate 11th grade math, group by school, then calculate & round average
eleventh_math = math_by_grade[(math_by_grade["grade"] == "11th")]
eleventh_math_grouped = eleventh_math.groupby("school_name")
eleventh_math_avg = round(eleventh_math_grouped.mean(),2)

# Separate 12th grade math, group by school, then calculate & round average
twelfth_math = math_by_grade[(math_by_grade["grade"] == "12th")]
twelfth_math_grouped = twelfth_math.groupby("school_name")
twelfth_math_avg = round(twelfth_math_grouped.mean(),2)

# Merge 9 and 10, then rename columns to be descriptive
nine_ten_math_merge = pd.merge(nineth_math_avg,tenth_math_avg, on="school_name")
nine_ten_math_merge.rename(columns={"math_score_x":"9th Grade","math_score_y":"10th Grade"}, inplace=True)

# Merge those with 11 and 12, then rename columns to be descriptive
nine_eleven_math_merge = pd.merge(nine_ten_math_merge,eleventh_math_avg, on="school_name")
nine_twelve_math_merge = pd.merge(nine_eleven_math_merge,twelfth_math_avg, on="school_name")
nine_twelve_math_merge.rename(columns={"math_score_x":"11th Grade","math_score_y":"12th Grade"}, inplace=True)

# Drop the index name
nine_twelve_math_merge.index.name = None

# Display the dataframe
nine_twelve_math_merge


,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# --------------------------------------
# Summarize reading scores, based on grade (9, 10, 11, 12)
# *To match the image in the assignment, we wouldn't round to 2 decimal places
# *However, that's inconsistent reporting, so I left it in
# --------------------------------------

# Make a new dataframe, based on the original data
# Keep only the school_name, grade, & reading_score series
reading_by_grade = school_data_complete[["school_name", "grade", "reading_score"]]
reading_by_grade.set_index("school_name")

# Separate 9th grade reading, group by school, then calculate & round average
nineth_reading = reading_by_grade[(reading_by_grade["grade"] == "9th")]
nineth_reading_grouped = nineth_reading.groupby("school_name")
nineth_reading_avg = round(nineth_reading_grouped.mean(),2)

# Separate 10th grade reading, group by school, then calculate & round average
tenth_reading = reading_by_grade[(reading_by_grade["grade"] == "10th")]
tenth_reading_grouped = tenth_reading.groupby("school_name")
tenth_reading_avg = round(tenth_reading_grouped.mean(),2)

# Separate 11th grade reading, group by school, then calculate & round average
eleventh_reading = reading_by_grade[(reading_by_grade["grade"] == "11th")]
eleventh_reading_grouped = eleventh_reading.groupby("school_name")
eleventh_reading_avg = round(eleventh_reading_grouped.mean(),2)

# Separate 12th grade reading, group by school, then calculate & round average
twelfth_reading = reading_by_grade[(reading_by_grade["grade"] == "12th")]
twelfth_reading_grouped = twelfth_reading.groupby("school_name")
twelfth_reading_avg = round(twelfth_reading_grouped.mean(),2)

# Merge 9 and 10, then rename columns to be descriptive
nine_ten_reading_merge = pd.merge(nineth_reading_avg,tenth_reading_avg, on="school_name")
nine_ten_reading_merge.rename(columns={"reading_score_x":"9th Grade","reading_score_y":"10th Grade"}, inplace=True)

# Merge those with 11 and 12, then rename columns to be descriptive
nine_eleven_reading_merge = pd.merge(nine_ten_reading_merge,eleventh_reading_avg, on="school_name")
nine_twelve_reading_merge = pd.merge(nine_eleven_reading_merge,twelfth_reading_avg, on="school_name")
nine_twelve_reading_merge.rename(columns={"reading_score_x":"11th Grade","reading_score_y":"12th Grade"}, inplace=True)

# Drop the index name
nine_twelve_reading_merge.index.name = None

# Display the dataframe
nine_twelve_reading_merge


,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# --------------------------------------
# Summarize schools, based on per-student-spending
# *Note: GitHub doesn't parse these labels well, probably due to escape cahracters?
# --------------------------------------

# Make a copy of our school_summary dataframe (the unformatted one)
spend_ranges_df = school_summary_df.copy()

# Make some bins & labels to describe the data
spend_bins = [0,583,629,644,675]
spend_labels = ["< $584","$585-629","$630-644","$645-675"]

# Cut the data into a new series, remove unnecessary series, & round average
spend_ranges_df["Spending Ranges (Per Student)"] = pd.cut(spend_ranges_df["Per Student Budget"], spend_bins, labels=spend_labels, include_lowest=True)
spend_ranges_df = spend_ranges_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing","Spending Ranges (Per Student)"]]
spend_ranges_df = round(spend_ranges_df.groupby("Spending Ranges (Per Student)").mean(),2)

# Display the dataframe
spend_ranges_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $584,83.45,83.94,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# --------------------------------------
# Give a summary of schools, based on school size
# *To match the image in the assignment, we wouldn't round to 2 decimal places
# *However, that's inconsistent reporting, so I left it in
# --------------------------------------

# Make another copy of our school_summary dataframe (the unformatted one)
size_ranges_df = school_summary_df.copy()

# Make some bins & labels to describe the data
size_bins = [0,999,2000,5000]
size_labels = ["Small (<1000)","Medium (100-2000)","Large (2000-5000)"]

# Cut the data into a new series, remove unnecessary series, & round average
size_ranges_df["School Size"] = pd.cut(size_ranges_df["Total Students"], size_bins, labels=size_labels, include_lowest=True)
size_ranges_df = size_ranges_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing","School Size"]]
size_ranges_df = round(size_ranges_df.groupby("School Size").mean(),2)

# Display the dataframe
size_ranges_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (100-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.28


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
# --------------------------------------
# Summarize schools, based on type of school (charter or district)
# *To match the image in the assignment, we wouldn't round to 2 decimal places
# *However, that's inconsistent reporting, so I left it in
# --------------------------------------

# Make yet another copy of our school_summary dataframe (the unformatted one)
type_ranges_df = school_summary_df.copy()

# Remove unnecessary series & reset the index
type_ranges_df = type_ranges_df[["School type","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
type_ranges_df.set_index("School type", inplace=True)

# Group it all, get the averages, & round
type_ranges_df = round(type_ranges_df.groupby("School type").mean(),2)

# Display the dataframe
type_ranges_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


In [11]:
# Not in homework
# Another interesting thing to look at is average $/student based on school type

# Copy of our school_summary dataframe (the unformatted one)
type_student_budget_df = school_summary_df.copy()

# Remove unnecessary series & reset the index
type_student_budget_df = type_student_budget_df[["School type","Per Student Budget"]]

# Group it all, get the averages, & round
type_student_budget_df = type_student_budget_df.groupby("School type").mean()

# Format series to look cleaner
type_student_budget_df["Per Student Budget"] = type_student_budget_df["Per Student Budget"].map("${:,.2f}".format)

# Display the dataframe
type_student_budget_df.head()


,Per Student Budget
School type,
Charter,$599.50
District,$643.57
